In [37]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import ETFs as etfs

In [38]:
data = pd.read_csv('assets.csv', parse_dates=['Date'])
print(data.columns.tolist())
data.set_index('Date', inplace=True)
display(data)


['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume']


,Ticker,Close,High,Low,Open,Volume
Date,,,,,,
2012-01-03,CORN,42.619999,42.990002,42.180000,42.730000,114100.0
2012-01-03,DBO,26.042213,26.094877,25.770118,25.796449,318300.0
2012-01-03,EWA,12.307327,12.368641,12.234866,12.240439,2787500.0
2012-01-03,EWC,20.625572,20.648155,20.384690,20.437383,6790400.0
2012-01-03,EWG,14.720882,14.838239,14.706213,14.742887,3892700.0
...,...,...,...,...,...,...
2025-03-28,VDNR.L,136.095001,138.505005,136.065002,137.910004,31820.0
2025-03-28,VGK,70.980003,71.260002,70.820000,71.190002,6081100.0
2025-03-28,VNQ,89.709999,90.279999,89.129997,90.120003,3602900.0


In [39]:
data_close = data[['Ticker', 'Close']].copy()
display(data_close)
# data_close[data_close['Ticker'] == 'SPY']['Close'].plot()

,Ticker,Close
Date,,
2012-01-03,CORN,42.619999
2012-01-03,DBO,26.042213
2012-01-03,EWA,12.307327
2012-01-03,EWC,20.625572
2012-01-03,EWG,14.720882
...,...,...
2025-03-28,VDNR.L,136.095001
2025-03-28,VGK,70.980003
2025-03-28,VNQ,89.709999


In [40]:
close_wide = data_close.pivot(columns='Ticker', values='Close')
display(close_wide)

Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,42.619999,26.042213,NaN,NaN,12.307327,20.625572,14.720882,...,28.830000,100.450668,84.510712,317.519989,69.905655,NaN,27.664728,34.591949,33.707817,6.78375
2012-01-04,NaN,NaN,NaN,42.509998,26.165096,NaN,NaN,12.452251,20.587931,14.581521,...,28.389999,100.608231,83.505913,318.160004,70.114944,NaN,27.328693,34.000526,33.888031,6.69750
2012-01-05,NaN,NaN,NaN,41.509998,25.814003,NaN,NaN,12.290605,20.459963,14.412819,...,28.510000,100.876099,83.357315,313.359985,69.817879,NaN,26.831108,34.319885,33.472137,6.68250
2012-01-06,NaN,NaN,NaN,41.349998,25.919333,NaN,NaN,12.128961,20.249195,14.148768,...,27.910000,100.616127,84.015366,313.760010,69.392532,NaN,26.462755,34.207520,33.070107,6.72625
2012-01-09,NaN,NaN,NaN,42.189999,25.831558,NaN,NaN,12.167976,20.309416,14.207449,...,28.129999,100.860344,83.866692,312.640015,69.655838,NaN,26.663080,34.089237,33.097828,6.69625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,38.276001,10.405,100.550003,18.930000,14.140000,107.930000,56.490002,23.700001,41.509998,38.029999,...,29.959999,574.080017,89.448509,74.879997,129.030991,140.270004,71.730003,90.728004,74.690002,8.28750
2025-03-25,38.380001,10.440,100.720001,18.770000,14.030000,107.330002,56.580002,23.809999,41.709999,38.380001,...,30.580000,575.460022,89.438553,74.820000,129.460007,140.479996,72.160004,89.709999,75.089996,8.14875
2025-03-26,37.854000,10.400,98.169998,18.559999,14.110000,105.839996,56.080002,23.709999,41.380001,37.560001,...,30.530001,568.590027,88.850662,75.419998,130.250000,139.595001,71.139999,90.059998,74.410004,8.14500


In [41]:
used_data = close_wide.copy()
clean_data = used_data.fillna(method='bfill')

clean_data = clean_data.dropna()
display(clean_data)


clean_data['AIEQ']
print(clean_data['AIEQ'].isna().sum())

C:\Users\Shelly\AppData\Local\Temp\ipykernel_2012\3466450363.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  clean_data = used_data.fillna(method='bfill')


Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,21.902441,6.246624,40.000000,42.619999,26.042213,22.380074,42.757004,12.307327,20.625572,14.720882,...,28.830000,100.450668,84.510712,317.519989,69.905655,42.300961,27.664728,34.591949,33.707817,6.78375
2012-01-04,21.902441,6.246624,40.000000,42.509998,26.165096,22.380074,42.757004,12.452251,20.587931,14.581521,...,28.389999,100.608231,83.505913,318.160004,70.114944,42.300961,27.328693,34.000526,33.888031,6.69750
2012-01-05,21.902441,6.246624,40.000000,41.509998,25.814003,22.380074,42.757004,12.290605,20.459963,14.412819,...,28.510000,100.876099,83.357315,313.359985,69.817879,42.300961,26.831108,34.319885,33.472137,6.68250
2012-01-06,21.902441,6.246624,40.000000,41.349998,25.919333,22.380074,42.757004,12.128961,20.249195,14.148768,...,27.910000,100.616127,84.015366,313.760010,69.392532,42.300961,26.462755,34.207520,33.070107,6.72625
2012-01-09,21.902441,6.246624,40.000000,42.189999,25.831558,22.380074,42.757004,12.167976,20.309416,14.207449,...,28.129999,100.860344,83.866692,312.640015,69.655838,42.300961,26.663080,34.089237,33.097828,6.69625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,38.276001,10.405000,100.550003,18.930000,14.140000,107.930000,56.490002,23.700001,41.509998,38.029999,...,29.959999,574.080017,89.448509,74.879997,129.030991,140.270004,71.730003,90.728004,74.690002,8.28750
2025-03-25,38.380001,10.440000,100.720001,18.770000,14.030000,107.330002,56.580002,23.809999,41.709999,38.380001,...,30.580000,575.460022,89.438553,74.820000,129.460007,140.479996,72.160004,89.709999,75.089996,8.14875
2025-03-26,37.854000,10.400000,98.169998,18.559999,14.110000,105.839996,56.080002,23.709999,41.380001,37.560001,...,30.530001,568.590027,88.850662,75.419998,130.250000,139.595001,71.139999,90.059998,74.410004,8.14500


0


In [42]:
first_prices = clean_data.iloc[0] 
print(first_prices)

normalized_close = clean_data / first_prices

normalized_array = normalized_close.to_numpy(dtype='float32')

display(normalized_close)

Ticker
AIEQ        21.902441
ASHR.L       6.246624
BCHN.L      40.000000
CORN        42.619999
DBO         26.042213
EDEN        22.380074
EMXC        42.757004
EWA         12.307327
EWC         20.625572
EWG         14.720882
EWH         10.451927
EWJ         30.004278
EWQ         14.192593
EWS         13.147454
EWU         19.759277
EWW         41.377037
GLD        155.919998
IDUP.L      13.738456
IEMG        35.379612
INDA        23.198488
KSA         20.759974
PSCC         8.678681
SEMI.AS      5.195000
SLV         28.830000
SPY        100.450668
TLT         84.510712
USO        317.519989
VDE         69.905655
VDNR.L      42.300961
VGK         27.664728
VNQ         34.591949
VPL         33.707817
XMAF.L       6.783750
Name: 2012-01-03 00:00:00, dtype: float64


Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2012-01-04,1.000000,1.000000,1.000000,0.997419,1.004719,1.000000,1.000000,1.011775,0.998175,0.990533,...,0.984738,1.001569,0.988110,1.002016,1.002994,1.000000,0.987853,0.982903,1.005346,0.987286
2012-01-05,1.000000,1.000000,1.000000,0.973956,0.991237,1.000000,1.000000,0.998641,0.991971,0.979073,...,0.988900,1.004235,0.986352,0.986898,0.998744,1.000000,0.969867,0.992135,0.993008,0.985075
2012-01-06,1.000000,1.000000,1.000000,0.970202,0.995281,1.000000,1.000000,0.985507,0.981752,0.961136,...,0.968089,1.001647,0.994139,0.988158,0.992660,1.000000,0.956552,0.988887,0.981081,0.991524
2012-01-09,1.000000,1.000000,1.000000,0.989911,0.991911,1.000000,1.000000,0.988677,0.984672,0.965122,...,0.975720,1.004078,0.992379,0.984631,0.996426,1.000000,0.963793,0.985467,0.981904,0.987102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,1.747568,1.665700,2.513750,0.444158,0.542965,4.822594,1.321187,1.925682,2.012550,2.583405,...,1.039195,5.715044,1.058428,0.235828,1.845788,3.316000,2.592832,2.622807,2.215807,1.221669
2025-03-25,1.752316,1.671303,2.518000,0.440404,0.538741,4.795784,1.323292,1.934620,2.022247,2.607181,...,1.060701,5.728782,1.058310,0.235639,1.851925,3.320965,2.608376,2.593378,2.227673,1.201216
2025-03-26,1.728301,1.664899,2.454250,0.435476,0.541813,4.729207,1.311598,1.926495,2.006247,2.551478,...,1.058966,5.660391,1.051354,0.237528,1.863226,3.300043,2.571505,2.603496,2.207500,1.200663


In [43]:
# def create_X_y(data, window_size):
#     X, y = [], []
#     for i in range(len(data) - window_size):
#         X.append(data[i : i + window_size])
#         y.append(data[i + window_size])
#     return np.array(X), np.array(y)


def create_X_y(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data) - 1):
        X.append(data.iloc[i-window_size:i].values.flatten()) 
        y.append(data.iloc[i+1].values) 
    X = np.array(X)
    y = np.array(y)
    return X, y

# def create_X_y(data, window_size):
#     X, y = [], []
#     for i in range(window_size, len(data) - 1):
#         window = data.iloc[i-window_size:i]
#         target = data.iloc[i+1]
#         if (window.values < 0).any() or (target.values < 0).any():
#             continue
#         X.append(window.values.flatten()) 
#         y.append(target.values) 
#     return np.array(X), np.array(y)

window_size = 30
X, y = create_X_y(normalized_close, window_size)
print(X.shape)
print(y.shape)
display(X)

(3377, 990)
(3377, 33)


array([[1.        , 1.        , 1.        , ..., 1.0712941 , 1.06251269,
        1.14519994],
       [1.        , 1.        , 1.        , ..., 1.06035197, 1.05922217,
        1.1437258 ],
       [1.        , 1.        , 1.        , ..., 1.05402604, 1.06868145,
        1.13653952],
       ...,
       [1.88207333, 1.63793312, 2.85      , ..., 2.62280692, 2.21580657,
        1.22166948],
       [1.8771881 , 1.6493956 , 2.82549992, ..., 2.59337795, 2.22767307,
        1.20121618],
       [1.91932936, 1.64109522, 2.88649998, ..., 2.60349587, 2.20749992,
        1.20066341]])

In [44]:
dataset_size = len(X)
print(f"dataset_size: {dataset_size}")
train_size = int(dataset_size * 0.8)
val_size = dataset_size - train_size  - 1
test_size = 1

print(f"train_size: {train_size}, val_size: {val_size}, test_size: {test_size}")
print(f"dataset_size: {dataset_size}, window_size: {window_size}")

X_train, X_val, X_test = X[:train_size], X[train_size: train_size + val_size], X[train_size + val_size: ]
y_train, y_val, y_test= y[:train_size], y[train_size: train_size + val_size], y[train_size + val_size: ]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


dataset_size: 3377
train_size: 2701, val_size: 675, test_size: 1
dataset_size: 3377, window_size: 30


((2701, 990), (2701, 33), (675, 990), (675, 33), (1, 990), (1, 33))

In [45]:
rf = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=30,        
        max_depth=10,           
        max_features='sqrt',    
        n_jobs=-1,             
        random_state=42
    )
)
# rf = MultiOutputRegressor(
#     RandomForestRegressor(
#         n_estimators=100, 
#         random_state=42,
#         n_jobs=-1,
#     )
# )
rf.fit(X_train, y_train)


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=10,
                                                     max_features='sqrt',
                                                     n_estimators=30, n_jobs=-1,
                                                     random_state=42))

In [46]:
Y_pred = rf.predict(X_test)
y_pred_price = Y_pred * first_prices.values
y_pred_df = pd.DataFrame(y_pred_price, columns=used_data.columns)
display(y_pred_df)

Ticker,AIEQ,ASHR.L,BCHN.L,CORN,DBO,EDEN,EMXC,EWA,EWC,EWG,...,SLV,SPY,TLT,USO,VDE,VDNR.L,VGK,VNQ,VPL,XMAF.L
0,37.638658,11.929749,109.29308,21.733194,15.170608,99.495097,55.75988,22.124741,35.266694,29.579766,...,22.830973,425.168067,100.210489,75.9593,94.116395,106.135633,56.235321,93.312882,63.616168,7.979394


In [47]:
yesterday_prices = used_data.iloc[-2]  # second last row 

today_prices_actual = used_data.iloc[-1] # last row

today_prices_predicted = pd.Series(y_pred_price.flatten(), index=used_data.columns)


comparison_df = pd.DataFrame({
    'Yesterday_actual': yesterday_prices,
    'Today_actual': today_prices_actual,
    'Today_pred': today_prices_predicted,
})

tickers_names = used_data.columns

correct_list = []
for ticker in tickers_names:
    yesterday = yesterday_prices[ticker]
    actual_today = today_prices_actual[ticker]
    predicted = today_prices_predicted[ticker]

    sign_actual = np.sign(actual_today - yesterday)
    sign_pred = np.sign(predicted - yesterday)

    if sign_actual == sign_pred:
        correct_list.append(1)
    else:
        correct_list.append(0)
comparison_df['Correct'] = correct_list

count_of_ones = correct_list.count(1)
print(f"Count of correct predictions: {count_of_ones}")
print(f"accuracy: {count_of_ones / len(correct_list)}")
display(comparison_df)


Count of correct predictions: 22
accuracy: 0.6666666666666666


,Yesterday_actual,Today_actual,Today_pred,Correct
Ticker,,,,
AIEQ,37.658001,36.845001,37.638658,1
ASHR.L,10.460000,10.340000,11.929749,0
BCHN.L,97.080002,92.714996,109.293080,0
CORN,18.510000,18.459999,21.733194,0
DBO,14.130000,13.970000,15.170608,0
EDEN,105.330002,105.169998,99.495097,1
EMXC,56.110001,55.180000,55.759880,1
EWA,23.750000,23.469999,22.124741,1
EWC,41.250000,40.570000,35.266694,1
